In [ ]:
#
# Attempts to implement the following:
#
# H. Rave, V. Molchanov and L. Linsen, "Uniform Sample Distribution in Scatterplots via Sector-based Transformation," 
# 2024 IEEE Visualization and Visual Analytics (VIS), St. Pete Beach, FL, USA, 2024, pp. 156-160, 
# doi: 10.1109/VIS55277.2024.00039. 
# keywords: {Data analysis;Visual analytics;Clutter;Scatterplot de-cluttering;spatial transformation},
#

In [ ]:
import polars as pl
import numpy as np
from math import cos, sin, pi, sqrt, atan2
from shapely import Polygon
from   udist_scatterplots_via_sectors import UDistScatterPlotsViaSectors, xyUniformSampleDistributionSectorTransformDEBUG, xyUniformSampleDistributionSectorTransform
import time
import rtsvg
rt = rtsvg.RACETrack()
df = pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk1.csv')
df = rt.columnsAreTimestamps(df, 'parsedDate')
df = df.rename({'TimeSeconds':                '_del1_', 'parsedDate':                 'timestamp',
                'dateTimeStr':                '_del2_', 'ipLayerProtocol':            'pro',
                'ipLayerProtocolCode':        '_del3_', 'firstSeenSrcIp':             'sip',
                'firstSeenDestIp':            'dip',    'firstSeenSrcPort':           'spt',
                'firstSeenDestPort':          'dpt',    'moreFragments':              '_del4_',
                'contFragments':              '_del5_', 'durationSeconds':            'dur',
                'firstSeenSrcPayloadBytes':   '_del6_', 'firstSeenDestPayloadBytes':  '_del7_',
                'firstSeenSrcTotalBytes':     'soct',   'firstSeenDestTotalBytes':    'doct',
                'firstSeenSrcPacketCount':    'spkt',   'firstSeenDestPacketCount':   'dpkt',
                'recordForceOut':             '_del8_'})
df = df.drop([r'^_del\d_$'])
df = df.sample(2_000_000)
xy = rt.xy(df, x_field='soct', y_field='doct', color_by='dpt', align_pixels=False, dot_size='small')
xy._repr_svg_() # force a render to get the xy's to populate
_xvals_, _yvals_ = list(xy.df[xy.x_axis_col]), list(xy.df[xy.y_axis_col])
print(len(_xvals_), len(_yvals_))
_weights_        = [1.0] * len(_xvals_)
#xy

In [ ]:
import random
num_of_pts   = [40, 80, 25]
circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
colors       = ['#ff0000','#006400','#0000ff']
_xvals_, _yvals_, _weights_, _colors_ = [], [], [], []
for i in range(len(num_of_pts)):
    for j in range(num_of_pts[i]):
        a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
        x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
        _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])

#
# Simple Renderer
#
def renderSVG(xs, ys, colors, r=0.2):
    x0, y0, x1, y1 = min(xs), min(ys), max(xs), max(ys)
    xperc, yperc   = (x1-x0)*0.01, (y1-y0)*0.01
    x0, y0, x1, y1 = x0-xperc, y0-yperc, x1+xperc, y1+yperc
    svg = []
    svg.append(f'<svg x="0" y="0" width="256" height="256" viewBox="{x0} {y0} {x1-x0} {y1-y0}" xmlns="http://www.w3.org/2000/svg">')
    svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" x="0" y="0" fill="#ffffff" />')
    for i in range(len(xs)): svg.append(f'<circle cx="{xs[i]}" cy="{ys[i]}" r="{r}" fill="{colors[i]}" />')
    svg.append('</svg>')
    return ''.join(svg)

#
# xyJustVecs() - why not just use vectors?  Because it doesn't work...
# ... you need the expected density ... which means you need to divide the space into sectors
#
def xyJustVecs(xvals, yvals, weights=None, colors=None, iterations=4, vector_scalar=0.1):
    xvals_last, yvals_last = xvals, yvals
    for iters in range(iterations):
        xvals_next, yvals_next = [], []
        for i in range(len(xvals_last)):
            _x_, _y_ = xvals_last[i], yvals_last[i]
            _u_, _v_ = 0.0, 0.0
            for j in range(len(xvals_last)):
                if i != j:
                    _x_next_, _y_next_ = xvals_last[j], yvals_last[j]
                    _uv_ = rt.unitVector(((_x_, _y_), (_x_next_, _y_next_)))
                    _u_, _v_ = _u_ + _uv_[0], _v_ + _uv_[1]
            _uv_ = rt.unitVector(((0.0, 0.0),(_u_, _v_)))
            _x_next_, _y_next_ = _x_ - _uv_[0]*vector_scalar, _y_ - _uv_[1]*vector_scalar
            xvals_next.append(_x_next_), yvals_next.append(_y_next_)
        xvals_last, yvals_last = xvals_next, yvals_next
    return xvals_last, yvals_last
    
x_new, y_new = xyJustVecs(_xvals_, _yvals_, _weights_, _colors_, iterations=80, vector_scalar=0.1)
rt.tile([renderSVG(_xvals_, _yvals_, _colors_), renderSVG(x_new, y_new, _colors_)], spacer=10)

In [ ]:
x_new, y_new, svgs, svgs_for_sectors = xyUniformSampleDistributionSectorTransformDEBUG(rt, _xvals_, _yvals_, _weights_, _colors_, iterations=32, border_perc=0.1, vector_scalar=0.1)
#rt.table(svgs, per_row=8, spacer=10)

In [ ]:
_to_display_ = []
for i in range(1): _to_display_.append(random.choice(svgs_for_sectors))
rt.table(_to_display_, per_row=4, spacer=10)

In [ ]:
x_new, y_new = xyUniformSampleDistributionSectorTransform(rt, _xvals_, _yvals_, _weights_, iterations=4, border_perc=0.1, vector_scalar=0.1)
rt.tile([renderSVG(_xvals_, _yvals_, _colors_), renderSVG(x_new, y_new, _colors_, r=0.01)], spacer=10)

In [ ]:
#
# Area of a Triangle (for sector calculation)
# - so, easier if we fix the box to (0.0, 0.0) to (1.0, 1.0)
#   ... could then hard-code the segment intersection function
#
p0, p1, p2 = (0.2, 0.1),(0.9, 0.3),(0.5, 0.9)

# Using Heron's formula (requires sqrt)
a, b, c    = rt.segmentLength((p0,p1)), rt.segmentLength((p1,p2)), rt.segmentLength((p2,p0))
s          = (a+b+c)/2.0
_area_     = sqrt(s*(s-a)*(s-b)*(s-c))

# Using the cross product (least # of multiplies)
_area2_ = abs((p0[0]*(p1[1]-p2[1]) + 
               p1[0]*(p2[1]-p0[1]) + 
               p2[0]*(p0[1]-p1[1]))/2.0)

# Using the shoelace formula
_area3_ = abs((p0[0]*p1[1] - p1[0]*p0[1]) +
              (p1[0]*p2[1] - p2[0]*p1[1]) +
              (p2[0]*p0[1] - p0[0]*p2[1]))/2.0

print(_area_, _area2_, _area3_, Polygon([p0,p1,p2]).area)
Polygon([p0,p1,p2])

In [ ]:
#
# Demonstrates the shoelace formula on the corner sectors
#
_points_ = [(0.6,0.4),(0.0, 0.9),(0.0, 1.0),(0.2, 1.0)]
_sum_    = 0.0
for i in range(len(_points_)):
    p, q   = _points_[i], _points_[(i+1)%len(_points_)]
    _sum_ += p[0]*q[1] - q[0]*p[1] # p_x * q_y - q_x * p_y
_area_ = abs(_sum_)/2.0 # abs required if the order of the points is reversed

print(_area_, Polygon(_points_).area)
Polygon(_points_)

In [ ]:
#
# Demonstrates the shoelace formula on a non-corner sector (if the middle point is just included to make the calculation the same for all sectors)
# ... it may just be simpler to calculate the sector areas outside of the polars use for the sector summation...
#
_points_ = [(0.5,0.3),(0.0, 0.9),(0.0, 0.8),(0.0, 0.6)]
_sum_    = 0.0
for i in range(len(_points_)):
    p, q   = _points_[i], _points_[(i+1)%len(_points_)]
    _sum_ += p[0]*q[1] - q[0]*p[1]
_area_ = abs(_sum_)/2.0 # abs required if the order of the points is reversed

print(_area_, Polygon(_points_).area)
Polygon(_points_)

In [ ]:
#
# Entire Algorithm As Polars Operations
#
num_of_pts   = [160, 320, 100]
circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
colors       = ['#ff0000','#006400','#0000ff']
_xvals_, _yvals_, _weights_, _colors_ = [12.0], [8.0], [1.0], ['#000000']
for i in range(len(num_of_pts)):
    for j in range(num_of_pts[i]):
        a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
        x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
        _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])
df            = pl.DataFrame({'x':_xvals_, 'y':_yvals_, 'w':_weights_, 'c':_colors_}) # the 'c' field is not required ... it's for the visualization of the data
df_orig       = rt.copyDataFrame(df)
t0            = time.time()
vector_scalar = 0.01 # parameter for the adjustment
iterations    = 256  # parameter for the algorithm

udspvs = UDistScatterPlotsViaSectors(_xvals_, _yvals_, _weights_, _colors_, vector_scalar, iterations)
df     = udspvs.df_results

t1 = time.time()

#
# Performance
# Time Taken (s): 12.910s | Iterations: 128 | Points: 1450 | M1 Pro (16G)
# Time Taken (s): 12.693s | Iterations: 128 | Points: 1450 | M1 Pro (16G)
# Time Taken (s):  3.302s | Iterations:  32 | Points: 1450 | M1 Pro (16G)
# Time Taken (s):  7.364s | Iterations:  64 | Points: 1451 | M1 Pro (16G)
#
print(f'Time Taken (s): {t1-t0:.3f}s | Iterations: {iterations} | Points: {len(df)}')

rt.tile([rt.xy(df_orig, x_field='x', y_field='y', dot_size='small', color_by='c'),
         rt.xy(df,      x_field='x', y_field='y', dot_size='small', color_by='c')])

In [ ]:
#
# Sector Alignment Examples
#
for _sector_ in range(16):
    # Sector Table Calculation
    _a0_     = _sector_*pi/8.0
    _a1_     = (_sector_+1)*pi/8.0
    _a_      = (_a0_ + _a1_) / 2.0
    _u_, _v_ = cos(_a_), sin(_a_)
    # Sector Delta Calculation
    # df   = df.with_columns(((16*(pl.arctan2(_dy_, _dx_) + pl.lit(pi))/(pl.lit(2*pi))).cast(pl.Int64)).alias('sector'))
    _sector_other_ = int(16 * (atan2(_v_, _u_) + pi)/(2*pi))
    _sector_align_ = (_sector_ + 8)%16
    print(f'{_sector_:>3} | {_sector_other_:>3} | {_sector_align_:>3}')


In [ ]:
# Testing the sector alignment...
amin, amax = {}, {}
for _sector_ in range(16):
    # Sector Table Calculation
    _a0_     = _sector_*pi/8.0
    _a1_     = (_sector_+1)*pi/8.0
    amin[_sector_] = _a0_
    amax[_sector_] = _a1_

# No errors show up this way ... however, errors are present for some of the exact angles
# ... but i don't think enough errors to cause the current issue (2025-06-07 0915)
_errors_ = 0
for i in range(10_000):
    a              = random.random() * 360.0
    _rad_          = pi * a / 180.0
    _sector_       = None
    for _s_ in amin.keys():
        if _rad_ >= amin[_s_] and _rad_ < amax[_s_]:
            _sector_ = _s_
            break
    _u_, _v_       = cos(_rad_), sin(_rad_)
    _atan2_        = atan2(_v_, _u_)
    _sector_other_ = int(16 * (atan2(_v_, _u_) + pi)/(2*pi))
    _sector_align_ = (_sector_ + 8)%16
    #print(f'{a:>3} | {_sector_other_:>3} | {_sector_align_:>3} | {_sector_other_ - _sector_align_:>3}')
    if _sector_other_ != _sector_align_: _errors_ += 1

print(f'Errors: {_errors_}')

In [ ]:
_point_i_ = 0
rt.tile([udspvs.svgOfPoint(rt, _point_i_)]) # , rt.histogram(df_points_all.filter(pl.col('__index__') == _point_i_), bin_by='sector', color_by='sector', bar_h=20, h=768, w=384)])
#udspvs.df_fully_filled[0]

In [ ]:
_tiles_ = []
for i in range(16): _tiles_.append(udspvs.renderSector(rt, point_i=0, sector=i))
rt.table(_tiles_, per_row=8)

In [ ]:
rt.tile([udspvs.animateIterations(animation_dur="10.0s"), udspvs], spacer=10)